In [1]:
from deepface import DeepFace
from deepface.extendedmodels import  Gender
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from zipfile import ZipFile
import tarfile
from urllib.request import urlopen
from scipy.io import loadmat
import csv
import sys
import re

Using TensorFlow backend.


## Data extraction

In [2]:
#variables to be defined
dataset='imdb'
data_path='../Data/imdb_crop.tar'
output_path= '../Data/imdb/'

In [3]:
#### Data extraction:
def extract_data(dataset, data_path, output_path):
    d={'imdb':'', 'wiki':'gz'}
    if dataset=='imdb'or dataset=='wiki':
        tar = tarfile.open(data_path,"r:"+ d[dataset])
        tar.extractall(output_path)
        tar.close()
    else:
        with ZipFile(data_path, 'r') as zipObj:
        # Extract all the contents of zip file 
            zipObj.extractall(output_path) 

In [4]:
#if the data is one of 5 remaining, use this code to extract image_path
#extract_data(dataset, data_path, output_path)
images=[]
if len(next(os.walk(output_path))[1]) == 0: #case when images are in one folder
        for image in os.listdir(output_path): 
            if image[-3:] == 'jpg' or image[-3:] == 'png':
                images.append(output_path + image)
else: #case when images are in several folders
        for path in [output_path + next(os.walk(output_path))[1][n] for n in range(len(next(os.walk(output_path))[1]))]:
            if len(next(os.walk(path))[1]) == 0:
                for image in os.listdir(path+'/'): 
                    if image[-3:] == 'jpg' or image[-3:] == 'png':
                           images.append(path+'/' + image)
            else:
                for sub_path in [path+'/' + next(os.walk(path))[1][n] for n in range(len(next(os.walk(path))[1]))]:
                    for image in os.listdir(sub_path+'/'): 
                        if image[-3:] == 'jpg' or image[-3:] == 'png':
                               images.append(sub_path+'/' +image)
print('Total number of images:', len(images))

Total number of images: 460723


In [8]:
## Load the gender model from deepface
models = {}
models["gender"] = Gender.loadModel()


In [9]:
def deepface(img_path):
    deepface_pred = DeepFace.analyze(img_path, actions=['gender'] ,models=models,enforce_detection= True)
    if deepface_pred['gender']=='Woman':
        gender='f'
    else:
        gender='m'
    return gender

In [10]:
with open(output_path+dataset+'.csv', 'w', newline='') as myfile:             # Here output written in the file
        wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
        wr.writerow(['Image_name','Pred_Gender'])
        for imagelocation in images[:10]:                        # Get the images location
                try:
                    deepface_gender=deepface(imagelocation)   # return the predicted gender
                    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                    wr.writerow([imagelocation,deepface_gender])
                except:
                    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                    wr.writerow([imagelocation,'u'])   #if there is an error in returning gender , write undefined
                    print("Oops!",sys.exc_info()[0],"occured.")
                    #print()

Action: gender:   0%|          | 0/1 [00:00<?, ?it/s] 

Actions to do:  ['gender']
already built gender model is passed



Action: gender:   0%|          | 0/1 [00:00<?, ?it/s] 

Actions to do:  ['gender']
already built gender model is passed
Oops! <class 'ValueError'> occured.
Actions to do:  ['gender']
already built gender model is passed



Action: gender:   0%|          | 0/1 [00:00<?, ?it/s] 

...

Actions to do:  ['gender']
already built gender model is passed



Action: gender:   0%|          | 0/1 [00:00<?, ?it/s] 

Actions to do:  ['gender']
already built gender model is passed
Oops! <class 'ValueError'> occured.
Actions to do:  ['gender']
already built gender model is passed



Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

Actions to do:  ['gender']
already built gender model is passed
Oops! <class 'ValueError'> occured.


In [4]:
if dataset== 'twitter':
    df_twitter=pd.read_csv(output_path+dataset+'.csv')
    df_twitter['Image_name'] = df_twitter['Image_name'].apply(lambda x: x.split('/')[-1]) #removing the folder names
    df_twitter['Image_name']=df_twitter['Image_name'].apply(lambda x: x.split('.')[0])
    df_twitter.to_csv(output_path+dataset+'.csv', index= False)
if dataset=='Gender_Shade':
    df_shade=pd.read_csv(output_path+dataset+'.csv')
    df_shade['Image_name']= df_shade['Image_name'].apply(lambda x: x.split('/')[-1])
    df_shade.to_csv(output_path+dataset+'.csv', index= False)
if dataset=='OUI'or dataset=='wiki' or dataset=='imdb':
    df_oui=pd.read_csv(output_path+dataset+'.csv')
    df_oui['Image_name']= df_oui['Image_name'].apply(lambda x: x.split('/'))
    df_oui['Image_name']=df_oui['Image_name'].apply(lambda x: x[-2]+'/'+x[-1])
    df_oui['Image_name']=df_oui['Image_name'].apply(lambda x: re.sub('landmark_aligned_face.', '', x) ) 
    df_oui.to_csv(output_path+dataset+'.csv', index= False)
  

    

## Extract  annotation

In [16]:
## For Gender_shade dataset
shade_annotation= pd.read_csv('../Data/PPB-2017-metadata.csv')
shade_annotation= shade_annotation[['filename','gender']]
shade_annotation.columns = ['Image_name', 'true_gender']
shade_annotation['true_gender']= shade_annotation['true_gender'].apply(lambda x: 'f' if x=='Female' else 'm')
print(shade_annotation.head(2),'\n')
print('the shape of the annotation dataframe is',shade_annotation.shape)

        Image_name true_gender
0  0001_1_f_FL.jpg           f
1  0002_1_f_FL.jpg           f 

the shape of the annotation dataframe is (1270, 2)


In [ ]:
### For TWITTER dataset ###
# 1)Extract the annotations
#extract_data(dataset, '../Data/twitter/_a_results32langs.zip', '../Data/twitter/_a_results32langs/')
###
twitter_annotation = pd.DataFrame()
meta_path = '../Data/twitter/_a_results32langs/'
for file in os.listdir(meta_path):
    df = pd.read_csv(meta_path + file)
    twitter_annotation = twitter_annotation.append(df)
    
twitter_annotation.reset_index(inplace = True)


## FOR TWITTER ##
twitter_annotation=twitter_annotation[['temp_file','indicated_gender','indicated_gender:confidence']]
twitter_annotation.rename(columns={'temp_file':'Image_name', 'indicated_gender': 'true_gender'}, inplace=True)
twitter_annotation = twitter_annotation[twitter_annotation['indicated_gender:confidence'] >= 0.8][(twitter_annotation['true_gender']=='male') | (twitter_annotation['true_gender']=='female') | (twitter_annotation['true_gender']=='orga')]
twitter_annotation['true_gender'] = twitter_annotation['true_gender'].apply(lambda x: "f" if x == 'female' else ("m" if x=='male' else 'u') ) 
####
print(twitter_annotation.head(2),'\n')
print('the shape of the annotation dataframe is',twitter_annotation.shape)


In [ ]:
## for OUI Dataset
def oui_annotation_extraction(annotation_path,annotation_output_path):
    extract_data(dataset, annotation_path, annotation_output_path)  #unzip the file 
    files=[annotation_output_path+'/'+name for name in os.listdir(annotation_output_path) if os.path.isfile(os.path.join(annotation_output_path, name))]
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, sep="\t")  #load each file as dataframe
        df= df.append(data, ignore_index = True)
    df= df.astype(str)
    df['Image_name']=df['user_id']+'/'+ df['face_id']+'.'+df['original_image'] # make Image name similar to the one in the image path
    df= df[['Image_name','gender']] # keep only gender and imagename
    df=df.rename(columns={"gender": "true_gender"})
    df['true_gender']=df['true_gender'].apply(lambda x: 'u' if x=='nan' else x)
    return df.drop_duplicates()

In [ ]:
oui_annotation= oui_annotation_extraction('../Data/OUI_annotations.zip','../Data/OUI_annotations')
print(oui_annotation.head(2),'\n')
print('the shape of the annotation dataframe is',oui_annotation.shape)

In [14]:
## For IMDB and wiki datasets
############################
### for .mat file for IMDB ###
def get_names(x):
    if len(x)>0:
        return x[0]
    else:
        return ''
### for .mat file for imdb and wiki ###
if dataset == 'imdb' or dataset == 'wiki':
    path_to_meta = output_path+'imdb_crop'+ '/' + dataset + ".mat"
    mat = loadmat(path_to_meta)  # load mat-file
    mdata = mat[dataset]  # variable in mat file
    mdtype = mdata.dtype
    ndata = {n: mdata[n][0, 0] for n in mdtype.names}
    columns = [n for n, v in ndata.items()]# if v.size == ndata['numIntervals']] 
    full_path = [mdata['full_path'][0,0][0][n][0] for n in range(len(mdata['full_path'][0,0][0]))]
    gender = mdata['gender'][0,0][0]
    #gender_dic= {1:'m',0:'f'}
    # gender=list(map(gender_dic.get, gender))
    
    if dataset == 'imdb':
        imdb_annotation = pd.DataFrame({ "Image_name":full_path, "true_gender":gender})
        imdb_annotation['true_gender'] = imdb_annotation['true_gender'].apply(lambda x: "f" if x == 0 else "m")
    elif dataset=='wiki':
        wiki_annotation = pd.DataFrame({ "Image_name":full_path, "true_gender":gender})
        wiki_annotation['true_gender'] = wiki_annotation['true_gender'].apply(lambda x: "f" if x == 0 else "m")

print(imdb_annotation.head(2),'\n')
print('the shape of the annotation dataframe is',imdb_annotation.shape)

                                     Image_name true_gender
0   01/nm0000001_rm124825600_1899-5-10_1968.jpg           m
1  01/nm0000001_rm3343756032_1899-5-10_1970.jpg           m 

the shape of the annotation dataframe is (460723, 2)


## Evaluation

In [16]:
### Precision, recall, F1_score
## set the value of df_annotation (df_annotation= dataset's annotation file)
df_annotation= imdb_annotation
#read the result_file
df_pred=pd.read_csv(output_path+dataset+'.csv')

##merge the annotation with the model prediction
result_data= pd.merge(df_annotation,df_pred, how= 'inner', on='Image_name')
result_data.to_csv(output_path+'/all_result_'+dataset+'.csv')

#remove the undifined gender
result_data= result_data[result_data.Pred_Gender !='u']

# True values
y_true = result_data['true_gender']
# Predicted values
y_pred = result_data['Pred_Gender']

# Print the confusion matrix
print(confusion_matrix(y_true, y_pred))
# Print the precision and recall, among other metrics
print(classification_report(y_true, y_pred, digits=4))

#printing the metrics
metrics_dict=classification_report(y_true, y_pred,output_dict=True)

#precision:
print('Precision:',round(metrics_dict['weighted avg']['precision'],4))
#Recall
print('Recall:',round(metrics_dict['weighted avg']['recall'],4))
#F1-score
print('F1-score:',round(metrics_dict['weighted avg']['f1-score'],4))
#accuracy
print('Accuracy:',round(metrics_dict['accuracy'],4))

[[ 82728  33997]
 [ 22719 122456]]
              precision    recall  f1-score   support

           f     0.7845    0.7087    0.7447    116725
           m     0.7827    0.8435    0.8120    145175

    accuracy                         0.7834    261900
   macro avg     0.7836    0.7761    0.7783    261900
weighted avg     0.7835    0.7834    0.7820    261900

Precision: 0.7835
Recall: 0.7834
F1-score: 0.782
Accuracy: 0.7834


In [8]:
result_data

,Image_name,true_gender,Pred_Gender
0,01/nm0000001_rm124825600_1899-5-10_1968.jpg,m,m
1,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,m,m
2,01/nm0000001_rm577153792_1899-5-10_1968.jpg,m,m
5,02/nm0000002_rm1075631616_1924-9-16_1991.jpg,f,f
6,02/nm0000002_rm1346607872_1924-9-16_2004.jpg,f,f
...,...,...,...
461924,08/nm3994408_rm727691264_1989-12-29_2011.jpg,f,m
461925,08/nm3994408_rm73386752_1989-12-29_2011.jpg,f,f
461926,08/nm3994408_rm744468480_1989-12-29_2011.jpg,f,m
461928,08/nm3994408_rm761245696_1989-12-29_2011.jpg,f,f
